In [1]:
# file dall'anno 2008 al 2018
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

file = []
#file.append("D:/Bets/eurodata/all-euro-data-2008-2009.xls")
#file.append("D:/Bets/eurodata/all-euro-data-2009-2010.xls")
#file.append("D:/Bets/eurodata/all-euro-data-2010-2011.xls")
#file.append("D:/Bets/eurodata/all-euro-data-2011-2012.xls")
#file.append("D:/Bets/eurodata/all-euro-data-2012-2013.xls")
#file.append("D:/Bets/eurodata/all-euro-data-2013-2014.xls")
#file.append("D:/Bets/eurodata/all-euro-data-2014-2015.xls")
#file.append("D:/Bets/eurodata/all-euro-data-2015-2016.xls")
file.append("D:/Bets/eurodata/all-euro-data-2016-2017.xls")
file.append("D:/Bets/eurodata/all-euro-data-2017-2018.xlsx")
file.append("D:/Bets/eurodata/all-euro-data-2018-2019.xlsx")


In [2]:
# dati di 7 bookmakers presenti in ogni anno
df = pd.DataFrame(index=[],columns=[])

for i in range (0,len(file)):
    xls = pd.ExcelFile(file[i])
    for sheet_count in range(0,22):
        df = df.append(pd.read_excel(xls, sheet_name = sheet_count, usecols = ['FTR',
                                                                               'B365H', 'B365D', 'B365A', 
                                                                               'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 
                                                                               'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA',
                                                                              'PSH', 'PSD', 'PSA', 'PSCH', 'PSCD', 'PSCA']))

In [3]:
df['FTR'] = df['FTR'].replace('H',0)
df['FTR'] = df['FTR'].replace('D',1)
df['FTR'] = df['FTR'].replace('A',2)

In [4]:
# cancello le righe con elementi vuoti
ndf = df.dropna()

In [5]:
target = pd.DataFrame(index=[],columns=[])
target = target.append(ndf[['FTR']])

In [6]:
ndf = df.drop(['FTR'],axis=1)

In [7]:
# normalizzo il training dataset
from sklearn import preprocessing
norm_data = preprocessing.normalize(ndf)
norm_data = norm_data.astype('float')

In [8]:
norm_data_df = pd.DataFrame(norm_data, columns=ndf.columns)

In [9]:
#valuto il modello svc multiclasse con decision function shape one vs one

from sklearn import model_selection
from sklearn.svm import SVC
name = 'svm'
results = []
scoring = 'accuracy'
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model = SVC(kernel = 'rbf', cache_size = 1000, gamma = 'scale', shrinking = True, class_weight = 'balanced',
            decision_function_shape = 'ovo')
cv_results = model_selection.cross_val_score(model, norm_data_df, target, cv=kfold, scoring=scoring)
results.append(cv_results)
msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
print(msg)

C:\Users\Francesco\Miniconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Francesco\Miniconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Francesco\Miniconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Francesco\Miniconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of 

svm: 0.464276 (0.020076)


In [10]:
model.fit(norm_data_df,target)

C:\Users\Francesco\Miniconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=1000, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [11]:
# creo il dataframe su cui fare predizioni

df_p = pd.DataFrame(index=[],columns=[])

file_p = []
file_p.append("D:/Bets/eurodata/all-euro-data-2019-2020.xlsx")

xls_p = pd.ExcelFile(file_p[0])
for sheet_count in range(0,22):
    df_p = df_p.append(pd.read_excel(xls_p, sheet_name = sheet_count, usecols = ['FTR', 'B365H', 'B365D', 'B365A', 
                                                                                'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 
                                                                                'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA',
                                                                               'PSH', 'PSD', 'PSA', 'PSCH', 'PSCD', 'PSCA']))

In [12]:
df_p['FTR'] = df_p['FTR'].replace('H',0)
df_p['FTR'] = df_p['FTR'].replace('D',1)
df_p['FTR'] = df_p['FTR'].replace('A',2)

In [13]:
# cancello le righe con elementi vuoti
ndf_p = df_p.dropna()

In [14]:
# salvo in un dataframe a singola colonna i risultati per confrontarli con quelli predetti

target_p = pd.DataFrame(index=[],columns=[])
target_p = target_p.append(ndf_p[['FTR']])

In [15]:
ndf_p = ndf_p.drop(['FTR'],axis=1)

In [16]:
# normalizzo i dati 
npdata = preprocessing.normalize(ndf_p)
npdata = npdata.astype('float')
npdata_df = pd.DataFrame(npdata, columns = ndf_p.columns)

In [17]:
prediction = model.predict(npdata_df)

In [18]:
# resetto gli indici 
ntarget_p = target_p.reset_index()
NEWdf_p = ndf_p.reset_index()

In [26]:
correct = 0
oddsH = pd.DataFrame(index=[],columns=[])
HW = 0
oddsD = pd.DataFrame(index=[],columns=[])
D = 0
oddsA = pd.DataFrame(index=[],columns=[])
AW = 0

for i in range (len(prediction)):
    
    if (prediction[i] == ntarget_p.at[i,'FTR']):
        correct += 1
        
        if(prediction[i] == 0):
            HW += 1
            oddsH = oddsH.append([[i,NEWdf_p.loc[[i],['B365H', 'BWH', 'IWH', 'WHH', 'VCH', 'PSH', 'PSCH']].values.max(),
                                   NEWdf_p.loc[i,['B365H', 'BWH', 'IWH', 'WHH', 'VCH', 'PSH', 'PSCH']].idxmax(axis=1)]])
            
        if(prediction[i] == 1):
            D += 1
            oddsD = oddsD.append([[i,NEWdf_p.loc[[i],['B365D', 'BWD', 'IWD', 'WHD', 'VCD', 'PSD', 'PSCD']].values.max(),
                                   NEWdf_p.loc[i,['B365D', 'BWD', 'IWD', 'WHD', 'VCD', 'PSD', 'PSCD']].idxmax(axis=1)]])
        
        if(prediction[i] == 2):
            AW += 1
            oddsA = oddsA.append([[i,NEWdf_p.loc[[i],['B365A', 'BWA', 'IWA', 'WHA', 'VCA', 'PSA', 'PSCA']].values.max(),
                                   NEWdf_p.loc[i,['B365A', 'BWA', 'IWA', 'WHA', 'VCA', 'PSA', 'PSCA']].idxmax(axis=1)]])

In [27]:
# con un training set più grande diminuisce il numero di pareggi riconosciuto, aumentano HW e AW

print(HW,D,AW)

583 302 429


In [28]:
# somma delle quote delle partite categorizzate correttamente 
total_odds = oddsH[1].sum() + oddsD[1].sum() + oddsA[1].sum()
total_odds

2980.87

In [29]:
# quote medie
average = total_odds / correct
average

2.268546423135464

In [30]:
# percentuale di partite categorizzate correttamente 
perc = (correct / len(ntarget_p)) * 100
perc

45.41997926028344

In [31]:
perc * average 

103.03733148980297

In [32]:
correct

1314

In [25]:
# overfitting?
# i risultati ottenuti allenando il network coi soli dati del 2018 sono migliori rispetto a quelli per 6 anni con 5 bookmakers

# su 3 anni con 7 bookmakers ottengo risultati molto incoraggianti...ma è un caso o ha un senso?